In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [ ]:
dataName = 'WCup2022_TeamData.csv'
DFTeamData = pd.read_csv(dataName)
DFTeamData

In [ ]:
def Ranking(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    
    for team_index in Team_list:
        scored_sum = 0
        conceded_sum = 0
        winningPoint = 0
        for team_columns in Team_list:
            score = RoundRobin.at[team_index, team_columns]
            if type(score) is float:
                continue
            score_list = score.split(',')
            if len(score_list) != 2:
                continue
            
            scored, conceded = int(score_list[0]), int(score_list[1])
            scored_sum += scored
            conceded_sum += conceded
            if scored > conceded: winningPoint += 3
            elif scored == conceded: winningPoint += 1
                
        RoundRobin.at[team_index, 'WinningPoints'] = winningPoint
        RoundRobin.at[team_index, 'Score'] = scored_sum
        RoundRobin.at[team_index, 'Conceded'] = conceded_sum
        RoundRobin.at[team_index, 'GoalDiff'] = scored_sum - conceded_sum
        
    WinningPoints_rank = RoundRobin['WinningPoints'].rank(ascending=False, method='min')
    WinningPoints_list = WinningPoints_rank.tolist()
    RoundRobin['Ranking'] = WinningPoints_list
    Duplication_list1 = []
    for i in range(1,5):
        if WinningPoints_list.count(i) >= 2:
            Duplication_list1.append(i)
    for duplication1 in Duplication_list1:
        RoundRobin_duplication1 = RoundRobin.loc[RoundRobin['Ranking'] == duplication1]
        GoalDiff_rank = RoundRobin_duplication1['GoalDiff'].rank(ascending=False, method='min')
        GoalDiff_list = GoalDiff_rank.tolist()
        GoalDiff_index = GoalDiff_rank.index.tolist()
        for j in range(1,len(GoalDiff_list)+1):
            if GoalDiff_list.count(j) == 1:
                RoundRobin.at[GoalDiff_index[GoalDiff_list.index(j)], 'Ranking'] = j + duplication1-1
            elif GoalDiff_list.count(j) >= 2:
                idx = -1
                Duplication_list2 = []
                for cnt in range(GoalDiff_list.count(j)):
                    idx = GoalDiff_list.index(j, idx+1)
                    Duplication_list2.append(GoalDiff_index[idx])
                RoundRobin.loc[Duplication_list2, 'Ranking'] = j + duplication1-1
                RoundRobin_duplication2 = RoundRobin.loc[Duplication_list2]
                Score_rank = RoundRobin_duplication2['Score'].rank(ascending=False, method='min')
                Score_list = Score_rank.tolist()
                Score_index = Score_rank.index.tolist()
                for k in range(1,len(Score_list)+1):
                    if Score_list.count(k) == 1:
                        RoundRobin.at[Score_index[Score_list.index(k)], 'Ranking'] = k + j-1 + duplication1-1
    
    
    return RoundRobin

In [ ]:
def RankingProbability(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    #Score_list = ['5,0', '4,0', '3,0', '2,0', '1,0', '0,0', '0,1', '0,2', '0,3', '0,4', '0,5']
    #Score_list = ['1,0', '0,0', '0,1']
    # 7パターンで420秒(7分), 1秒で約240回
    # 予測：5試合分　11パターンで11分x11=2時間, 13パターンで26分x13=5時間38分
    # 予測：4試合分　19パターンで9分,25パターンで28分
    Score_list        = ['0,0', '1,1', '2,2', '3,0', '2,1', '2,0', '1,0', '0,1', '0,2', '1,2', '0,3']
    Score_inverselist = ['0,0', '1,1', '2,2', '0,3', '1,2', '0,2', '0,1', '1,0', '2,0', '2,1', '3,0']
    X = np.array(range(len(Score_list)))
    
    Ranking_num = pd.DataFrame(0, columns=range(1,5), index=Team_list)
    Ranking_probability = pd.DataFrame(columns=range(1,5), index=Team_list)
    print("start : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
    continueSwitch1 = 0
    for i1 in X:
        if continueSwitch1 == 1: continue
        print("0,1(" + str(i1) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
        RoundRobin1 = RoundRobin.copy()
        if RoundRobin1.iat[0,1] is float(np.nan):
            RoundRobin1.iat[0,1] = Score_list[i1]
            RoundRobin1.iat[1,0] = Score_inverselist[i1]
        else: continueSwitch1 = 1
        continueSwitch2 = 0
        for i2 in X:
            if continueSwitch2 == 1: continue
            #print("0,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
            RoundRobin2 = RoundRobin1.copy()
            if RoundRobin2.iat[0,2] is float(np.nan):
                RoundRobin2.iat[0,2] = Score_list[i2]
                RoundRobin2.iat[2,0] = Score_inverselist[i2]
            else: continueSwitch2 = 1
            continueSwitch3 = 0
            for i3 in X:
                if continueSwitch3 == 1: continue
                #print("0,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                RoundRobin3 = RoundRobin2.copy()
                if RoundRobin3.iat[0,3] is float(np.nan):
                    RoundRobin3.iat[0,3] = Score_list[i3]
                    RoundRobin3.iat[3,0] = Score_inverselist[i3]
                else: continueSwitch3 = 1
                continueSwitch4 = 0
                for i4 in X:
                    if continueSwitch4 == 1: continue
                    #print("1,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                    RoundRobin4 = RoundRobin3.copy()
                    if RoundRobin4.iat[1,2] is float(np.nan):
                        RoundRobin4.iat[1,2] = Score_list[i4]
                        RoundRobin4.iat[2,1] = Score_inverselist[i4]
                    else: continueSwitch4 = 1
                    continueSwitch5 = 0
                    for i5 in X:
                        if continueSwitch5 == 1: continue
                        #print("1,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                        RoundRobin5 = RoundRobin4.copy()
                        if RoundRobin5.iat[1,3] is float(np.nan):
                            RoundRobin5.iat[1,3] = Score_list[i5]
                            RoundRobin5.iat[3,1] = Score_inverselist[i5]
                        else: continueSwitch5 = 1
                        continueSwitch6 = 0
                        for i6 in X:
                            if continueSwitch6 == 1: continue
                            #print("2,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                            RoundRobin6 = RoundRobin5.copy()
                            if RoundRobin6.iat[2,3] is float(np.nan):
                                RoundRobin6.iat[2,3] = Score_list[i6]
                                RoundRobin6.iat[3,2] = Score_inverselist[i6]
                            else: continueSwitch6 = 1
                                
                            RoundRobin_rank = Ranking(RoundRobin6.copy())
                            
                            for j in range(1,5):
                                for k in range(0,4):
                                    if RoundRobin_rank.at[Team_list[k], 'Ranking'] == j:
                                        Ranking_num.at[Team_list[k], j] += 1
                            
    
    for k in range(0,4):
        patternNum = Ranking_num.loc[Team_list[k], :].sum()
        for j in range(1,5):
            Ranking_probability.at[Team_list[k], j] = str((Ranking_num.at[Team_list[k], j]/patternNum*100).round(1)) + '%'
                            
    return Ranking_num, Ranking_probability

In [ ]:
Group_E_Team = DFTeamData.loc[DFTeamData['Group'] == 'E']
Group_E_Team_list = Group_E_Team['Team'].tolist()
Group_E_RoundRobin = pd.DataFrame(columns=Group_E_Team_list, index=Group_E_Team_list)
for team in Group_E_Team_list:
    Group_E_RoundRobin.at[team, team] = "-"

In [ ]:
Ranking_JapanPropability = pd.DataFrame(columns=range(1,5))

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,0' , '0,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,1' , '1,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['1,1', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '2,2' , '2,2'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['2,2', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,3' , '3,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['3,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['2,1', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['2,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,1' , '1,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['1,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,0' , '0,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,1', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '2,0' , '0,2'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,2', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '2,1' , '1,2'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['1,2', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '3,0' , '0,3'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,3', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

In [ ]:
Ranking_JapanPropability